In [1]:
import okx.MarketData as MarketData
import pandas as pd
from datetime import datetime
import pytz
import time
import os
class MarketDataFetcher:
    def __init__(self, flag="0", inst_id="BTC-USDT", bar="1m"):
        # 初始化 MarketAPI
        self.marketDataAPI = MarketData.MarketAPI(flag=flag)  # 实盘: 0, 模拟盘: 1
        self.inst_id = inst_id
        self.bar = bar

    def fetch_data(self, start_time, end_time):
        # 获取起始时间戳和结束时间戳
        start_timestamp, end_timestamp = self._get_time_stamps(start_time, end_time)
        if self.bar == '1m':
            time_interval = 60 
        # 获取起始时间戳和结束时间戳
        start_timestamp, end_timestamp = self._get_time_stamps(start_time, end_time)

        # 计算 total_data_count
        total_data_count = (end_timestamp - start_timestamp)/1000 // time_interval

        print(f"总数据条数: {total_data_count}")


        # 获取历史K线数据
        all_data, now_timestamp = [], end_timestamp
        while now_timestamp > start_timestamp:
            result = self.marketDataAPI.get_history_candlesticks(
                instId=self.inst_id, bar=self.bar, after=str(now_timestamp), limit="100"
            )
            # 清空控制台
            os.system('cls')
            data = result.get("data", [])
            all_data.extend(data)
            if data:
                now_timestamp = int(data[-1][0])  # 更新为最新的时间戳
        
                # 计算当前进度
                current_progress = len(all_data) / total_data_count * 100
                print(f"当前进度: {self.inst_id}：{current_progress:.2f}%")
                
                # time.sleep(0.1)  # 延时100ms
            else:
                break

        # 将数据转换为DataFrame并处理时间戳
        df = pd.DataFrame(all_data, columns=["timestamp", "open", "high", "low", "close", "vol", "volCcy", "volCcyQuote", "confirm"])
        df["timestamp"] = pd.to_numeric(df["timestamp"], errors='coerce')
        df["timestamp"] = df["timestamp"].apply(lambda x: self._convert_to_china_timezone(x))

        # 将K线数据转换为数值类型
        df[["open", "high", "low", "close", "vol", "volCcy", "volCcyQuote"]] = df[["open", "high", "low", "close", "vol", "volCcy", "volCcyQuote"]].apply(pd.to_numeric)
        
        # 按照 'timestamp' 列从前往后（升序）排序
        df = df.sort_values(by="timestamp", ascending=True)

        return df

    def _get_time_stamps(self, start_time, end_time):
        # 获取时间戳
        start_timestamp = int(start_time.timestamp() * 1000)
        end_timestamp = int(end_time.timestamp() * 1000)
        return start_timestamp, end_timestamp

    def _convert_to_china_timezone(self, timestamp):
        # 转换为北京时间（UTC+8）
        china_tz = pytz.timezone('Asia/Shanghai')
        return datetime.utcfromtimestamp(timestamp / 1000).replace(tzinfo=pytz.utc).astimezone(china_tz)

def save_data_to_excel(df, filename):
    # 去除时区信息并保存到Excel
    df["timestamp"] = df["timestamp"].dt.tz_localize(None)
    df.to_excel(filename, index=False)
    print(f"Data saved to '{filename}'.")

def calculate_time_diff(df):
    # 计算时间差并打印时间差信息
    df["time_diff"] = df["timestamp"].diff().dt.total_seconds()
    if not df.empty:
        print(f"第一个时间戳: {df['timestamp'].iloc[0]} (北京时间)")
        print(f"最后一个时间戳: {df['timestamp'].iloc[-1]} (北京时间)")
        
        time_diff_values = df["time_diff"].dropna().unique()
        if len(time_diff_values) == 1:
            print(f"所有时间差完全一致，时间差为 {time_diff_values[0]} 秒。")
        else:
            print(f"时间差有不同的值，存在 {len(time_diff_values)} 种不同的时间差值。")
            print(f"不同的时间差值: {time_diff_values}")
    else:
        print("DataFrame为空，无法处理时间戳或时间差。")


In [3]:
from datetime import datetime
import os
# 设置目标时间范围并转换为datetime对象
start_time_str = '2025_01_01_00_00_00'
# start_time_str = '2025_02_23_12_00_00'
end_time_str   = '2025_02_23_14_00_00'
start_time = datetime.strptime(start_time_str, '%Y_%m_%d_%H_%M_%S')
end_time = datetime.strptime(end_time_str, '%Y_%m_%d_%H_%M_%S')


# 确保目标文件夹存在
os.makedirs("data", exist_ok=True)


# 定义单个任务处理函数
def single_task(flag, inst_id, bar, start_time, end_time):
    # 创建MarketDataFetcher实例并获取数据
    market_data_fetcher = MarketDataFetcher(flag=flag, inst_id=inst_id, bar=bar)
    df = market_data_fetcher.fetch_data(start_time, end_time)

    # 保存数据到Excel，确保文件名格式正确
    filename = os.path.join("data", f"{inst_id}_history_data_{start_time_str}_to_{end_time_str}.xlsx")
    save_data_to_excel(df, filename)

    # 计算并打印时间差
    calculate_time_diff(df)

# 配置参数
inst_id_list = [
    # "BTC-USDT", "ETH-USDT", "SOL-USDT", "DOGE-USDT", "PEPE-USDT",
    # "OKB-USDT", "TON-USDT", "XRP-USDT", "BCH-USDT", "LTC-USDT"
    "BTC-USDT", "ETH-USDT", "DOGE-USDT", "PEPE-USDT"
    
]
bar = "1m"

# 批量获取并处理数据
for inst_id in inst_id_list:
    single_task(flag="0", inst_id=inst_id, bar=bar, start_time=start_time, end_time=end_time)


总数据条数: 77160.0
header:  {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/market/history-candles?instId=BTC-USDT&after=1740290400000&bar=1m&limit=100
当前进度: BTC-USDT：0.13%
header:  {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/market/history-candles?instId=BTC-USDT&after=1740284400000&bar=1m&limit=100
当前进度: BTC-USDT：0.26%
header:  {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/market/history-candles?instId=BTC-USDT&after=1740278400000&bar=1m&limit=100
当前进度: BTC-USDT：0.39%
header:  {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/market/history-candles?instId=BTC-USDT&after=1740272400000&bar=1m&limit=100
当前进度: BTC-USDT：0.52%
header:  {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/market/history-candles

KeyboardInterrupt: 